### ANN
- 
- 구성 : 전결합층(FC Layer)을 컬렉션(시퀀스)으로 구성
- 학습 : 회귀

[1] 모듈 로딩 및 데이터 준비

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [13]:
### 데이터 준비
DATA_FILE='./iris.csv'

### 데이터 로딩
### Header 파라미터 : 데이터 파일의 첫번째 줄이 컬럼명 여부 체크
### usecols 파라미터 : 로딩하고 싶은 컬럼 지정
iris_DF = pd.read_csv(DATA_FILE, header=None, usecols=[0,1,2,3])
iris_DF.head(5)

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


[2] 모델 클래스 설계 및 정의
- 전결합층 클래스 => DenseModel
- 컬렉션 클래스 => SeqenceModel

In [6]:
### 전결합층 클래스 

class DenseModel(tf.Module):
    def __init__(self, input_nums, out_nums):
        super().__init__()
        self.W = tf.Variable(tf.random.normal((input_nums, out_nums)))
        self.b = tf.Variable(tf.zeros((out_nums)))
    
    def __call__(self, input):
        wbsum = tf.matmul(input, self.W) + self.b

        return wbsum

In [7]:
### -------------------------------------------------------
### 컬렉션 클래스
### 클래스 목적 : 모델의 층 구성 클래스
### 클래스 이름 : SequenceModel
### 클래스 속성 : in_layer, d_layer, out_layer
### 클래스 기능 : 층별 학습 진행 후 결과 전달 즉, 전방향 학습 기능
### -------------------------------------------------------

class SequenceModel(tf.Module):
    #인스턴스 생성 메서드
    def __init__(self, in_nums, out_nums):
        super().__init__()
        
        #입력층 : 피쳐개수, 퍼셉트론개수
        self.in_layer = DenseModel(in_nums, in_nums*2)

        #은닉층 : 피쳐개수, 퍼셉트론개수
        self.hd_layer = DenseModel(in_nums, in_nums*2, in_nums)
        
        #출력층 : 피쳐개수, 타켓개수
        self.out_layer = DenseModel(in_nums,out_nums)

    # 순방향/전방향 학습 메서드
    def __call__(self, data):
        #입력층 학습
        wbsum = self.in_layer(data)
        output = tf.nn.relu(wbsum)

        #은닉층 학습
        wbsum = self.hd_layer(output)
        output = tf.nn.relu(wbsum)

        #출력층 학습
        wbsum = self.hd_layer(output)

        return wbsum

[3] 학습 준비
- 데이터셋 -> 피쳐와 타켓 분리
- 인스턴스/객체 -> 모델 인스턴스, 최적화 인스턴스, 손실함수 인스턴스
- 학습 관련 함수들 -> 학습 함수, 검증함수, 예측함수

[3-1] 피쳐와 타켓 데이터셋 분리

In [14]:
### 텐서화 - 변경불가 데이터 즉, tf.constant()
feature_NP = iris_DF[ iris_DF.columns[:-1] ].values
target_NP = iris_DF[ iris_DF.columns[-1:] ].values

print(f'feature_NP shape -> {feature_NP.shape}, target_NP shape -> {target_NP.shape}')

feature_NP shape -> (150, 3), target_NP shape -> (150, 1)
